<h1><center>ASSIGNMENT 2</center></h1>
## TASK 1: Finding out Pearson correlation coefficient
### Selecting interesting variables
The *properties_2016.csv* file contains 58 columns of which one is the parcelid. A lot of the columns are mostly blank and so cannot be used for visualizing any meaningful relationships with other columns. The following code displays the rows that is at least 50% populated in each column. We will only use these columns to find out interesting relationships.

In [19]:
import pandas as pd
from scipy.stats import linregress
df = pd.read_csv("properties_2016.csv")
#regionidzip = df['bedroomcnt']
#calculatedfinishedsquarefeet= df['calculatedfinishedsquarefeet']
#df['bedroomcnt'].fillna(0, inplace=True)
#df['calculatedfinishedsquarefeet'].fillna(0, inplace=True)
#print(linregress(regionidzip,calculatedfinishedsquarefeet))
col_array=list(df)
data_array=(len(df.index)-df.isnull().sum())/len(df.index)
col_selected=[]
for i in range(0,len(data_array)):
    if data_array[i]>=0.5:
        col_selected.append(col_array[i])
        print(col_array[i],data_array[i])

/home/wasif/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


parcelid 1.0
bathroomcnt 0.996160413129
bedroomcnt 0.996164432937
buildingqualitytypeid 0.649362508655
calculatedbathnbr 0.956816539635
calculatedfinishedsquarefeet 0.981386612766
finishedsquarefeet12 0.907533355197
fips 0.99616878773
fullbathcnt 0.956816539635
heatingorsystemtypeid 0.605115474017
latitude 0.99616878773
longitude 0.99616878773
lotsizesquarefeet 0.907511246251
propertycountylandusecode 0.99588740115
propertylandusetypeid 0.99616878773
propertyzoningdesc 0.66280910232
rawcensustractandblock 0.99616878773
regionidcity 0.978947929079
regionidcounty 0.99616878773
regionidzip 0.995316923359
roomcnt 0.996156058337
unitcnt 0.662427555518
yearbuilt 0.979925077473
structuretaxvaluedollarcnt 0.981581908451
taxvaluedollarcnt 0.985746429824
assessmentyear 0.996168117762
landtaxvaluedollarcnt 0.977310527174
taxamount 0.989531749283
censustractandblock 0.974833990293


Of these columns, not all are interesting from a practical point of view. We certainly don't want to find the relation between a parcelid and the number of rooms the house have (we do keep the parcelid column for id-ing purposes)! Besides some of the columns contain certain official codes which we don't want as an interesting variable. So we take those columns out and have the final list of interesting variables.

In [21]:
remove_labels=['fips','propertyzoningdesc','rawcensustractandblock','censustractandblock','assessmentyear','taxdelinquencyyear']

for x in remove_labels:
    if x in col_selected:
        col_selected.remove(x)

for x in col_selected:
    print(x)
    
selected_df=df[col_selected].copy()

parcelid
bathroomcnt
bedroomcnt
buildingqualitytypeid
calculatedbathnbr
calculatedfinishedsquarefeet
finishedsquarefeet12
fullbathcnt
heatingorsystemtypeid
latitude
longitude
lotsizesquarefeet
propertycountylandusecode
propertylandusetypeid
regionidcity
regionidcounty
regionidzip
roomcnt
unitcnt
yearbuilt
structuretaxvaluedollarcnt
taxvaluedollarcnt
landtaxvaluedollarcnt
taxamount


### Finding out Pearson correlation coefficient
Now that we have identified the interesting variables, we would like to calculate the correlation between them. We do have some missing data in the interesting columns. We would be avoiding the missing information and use only the information present to calculate the pearson correlation coefficient.

In [22]:
selected_df.corr()

,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fullbathcnt,heatingorsystemtypeid,latitude,...,regionidcity,regionidcounty,regionidzip,roomcnt,unitcnt,yearbuilt,structuretaxvaluedollarcnt,taxvaluedollarcnt,landtaxvaluedollarcnt,taxamount
parcelid,1.000000,-0.041051,-0.031117,0.112804,-0.000757,0.002859,0.032189,-0.047473,0.201064,-0.185266,...,-0.033763,-0.454630,0.050051,0.432083,0.020472,0.172329,0.003220,0.004463,0.009355,-0.013466
bathroomcnt,-0.041051,1.000000,0.701141,-0.267869,1.000000,0.351715,0.738657,0.987360,-0.369936,0.012498,...,-0.027596,0.000994,0.017216,0.062613,0.037335,0.328097,0.273515,0.326395,0.281953,0.312825
bedroomcnt,-0.031117,0.701141,1.000000,-0.044097,0.629771,0.261357,0.573461,0.627372,-0.225345,0.028339,...,-0.011801,0.017734,0.007397,0.162882,0.055065,0.046866,0.127920,0.147426,0.115045,0.147094
buildingqualitytypeid,0.112804,-0.267869,-0.044097,1.000000,-0.264641,-0.139306,-0.224686,-0.264641,0.434448,-0.143521,...,0.038467,NaN,-0.013482,0.000811,0.096115,-0.436889,-0.114142,-0.063837,-0.024306,-0.062501
calculatedbathnbr,-0.000757,1.000000,0.629771,-0.264641,1.000000,0.684105,0.802892,0.987360,-0.369924,0.013954,...,-0.035911,-0.032926,0.022644,0.023051,0.248886,0.363602,0.570233,0.463487,0.344273,0.457954
calculatedfinishedsquarefeet,0.002859,0.351715,0.261357,-0.139306,0.684105,1.000000,1.000000,0.668983,-0.282097,-0.003483,...,-0.020837,-0.025239,0.014970,0.036318,0.387663,0.115244,0.629473,0.535773,0.323191,0.538527
finishedsquarefeet12,0.032189,0.738657,0.573461,-0.224686,0.802892,1.000000,1.000000,0.783162,-0.283622,-0.005215,...,-0.037076,-0.088586,0.034818,0.106040,0.081527,0.268415,0.704024,0.596757,0.455200,0.594913
fullbathcnt,-0.047473,0.987360,0.627372,-0.264641,0.987360,0.668983,0.783162,1.000000,-0.380933,0.057588,...,-0.032097,0.047390,0.017130,-0.027183,0.248848,0.321519,0.554208,0.449540,0.333334,0.447666
heatingorsystemtypeid,0.201064,-0.369936,-0.225345,0.434448,-0.369924,-0.282097,-0.283622,-0.380933,1.000000,-0.307144,...,0.038252,-0.501317,-0.006533,0.445865,0.006004,-0.343212,-0.188602,-0.132412,-0.083876,-0.134964
latitude,-0.185266,0.012498,0.028339,-0.143521,0.013954,-0.003483,-0.005215,0.057588,-0.307144,1.000000,...,-0.001134,0.569824,0.000981,-0.234332,-0.036381,-0.004852,0.002164,-0.060417,-0.104542,-0.042617
